$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n) (\sigma^x_{m} \sigma^x_{n} + \sigma^y_{m} \sigma^y_{n} + \sigma^z_{m} \sigma^z_n) $ where $J_{HS}(m-n) = \frac{\\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [2]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt

In [3]:
def wrap_index(N, m, n):
    if m >= n:
        swap = m
        m = n
        n = swap
    
    forward_dist = abs(m-n)
    back_dist = abs(N+m-n)
    return min(forward_dist, back_dist)

In [5]:
nqs_en_x = []
nqs_en_y = []
analytical_en_x = []
analytical_en_y = []
for N in range(3, 51):
    if N %2 == 0:
        hi = nk.hilbert.Spin(s=1/2, N=N)
        def haldane_shastry_model(N):
            sum = 0
            for n in range(N):
                for m in range(n):
                    hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(N, n, m))*np.pi/N)**2))
                    x = sigmax(hi, m) * sigmax(hi, n)
                    y = sigmay(hi, m) * sigmay(hi, n)
                    z = sigmaz(hi, m) * sigmaz(hi, n)
                    sum += hs * (x+y+z)
            return sum / 4
        hs20 = haldane_shastry_model(N)
        rbm = nk.models.RBM(alpha=1, param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))
        sampler = nk.sampler.MetropolisLocal(hi)
        vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)
        optimizer = nk.optimizer.Sgd(learning_rate=0.1)
        preconditioner = nk.optimizer.SR(diag_shift=0.1)
        vmc = nk.driver.VMC(hs20, optimizer, variational_state=vstate, preconditioner=preconditioner)
        log = nk.logging.RuntimeLog()
        vmc.run(n_iter=300, out=log)
        opt_energy = vstate.expect(hs20)
        nqs_en_x.append(N)
        analytical_en_x.append(N)
        nqs_en_y.append(opt_energy.mean)
        analytical_en_y.append(-np.pi**2/24*(N+5/N))
plt.plot(nqs_en_x, nqs_en_y, label="NQS") 
plt.plot(analytical_en_x, analytical_en_y, label="analytical") 
plt.legend()
    

/Users/adamremels/miniconda3/envs/csci1470/lib/python3.10/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples
  0%|          | 0/300 [00:00<?, ?it/s]/Users/adamremels/miniconda3/envs/csci1470/lib/python3.10/site-packages/netket/optimizer/qgt/qgt_onthefly.py:107: HolomorphicUndeclaredWarning: 
Defaulting to `holomorphic=False`, but this might lead to increased
computational cost or disabled features. Check if your variational
function is holomorphic, and if so specify `holomorphic=True`as an extra
keyword argument.

To silence this warning, specify the `holomorphic=False/True` keyword
argument.

To numerically check whether your variational function is or not holomorphic
you can use the following snippet:

```python
   vs = nk.vqs.MCState(...)

   nk.utils.is_probably_holomorphic(vs._apply_fun, vs.parameters, vs.samples, vs.model_state)

In [98]:
hs20 = haldane_shastry_model(N)

In [99]:
rbm = nk.models.RBM(alpha=1, param_dtype=float, kernel_init=nn.initializers.normal(stddev=0.01))
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)
optimizer = nk.optimizer.Sgd(learning_rate=0.1)
preconditioner = nk.optimizer.SR(diag_shift=0.1)
vmc = nk.driver.VMC(hs20, optimizer, variational_state=vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
vmc.run(n_iter=300, out=log)
opt_energy = vstate.expect(hs20)
print(opt_energy)

/Users/adamremels/miniconda3/envs/csci1470/lib/python3.10/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples
100%|██████████| 300/300 [00:08<00:00, 33.83it/s, Energy=-3.4491+0.0000j ± 0.0017 [σ²=0.0015, R̂=1.0217]]        


-3.4519+0.0000j ± 0.0017 [σ²=0.0015, R̂=1.0155]


In [100]:
true_energy = range(3,N+1)
true_energy = [np.pi**2/24*(x-5/x) for x in true_energy]
print(true_energy)



[0.5483113556160755, 1.1308921709581556, 1.6449340668482264, 2.1247065030122925, 2.584896390761499, 3.0328471857514177, 3.472638585568478, 3.906718408764538, 4.336644358054415, 4.763454901914656, 5.187868980059791, 5.610400120857343, 6.0314249117768295, 6.451225793420388, 6.870018749777887, 7.287971768397003, 7.705217471025902, 8.121861955063117]
